# Depreciation to FERC 1 Connection

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pudl
import pudl.constants as pc
import sqlalchemy as sa
import logging
import sys
import copy
import pathlib
import random
import warnings

sys.path.append("../")
import connect_deprish_to_eia
import make_plant_parts_eia
import connect_deprish_to_ferc1

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]
pd.options.display.max_columns = None

In [7]:
file_path_mul = pathlib.Path().cwd().parent / 'master_unit_list.pkl.gz'
file_path_steam_ferc1 = pathlib.Path().cwd().parent / 'steam_ferc1.pkl.gz'
file_path_ferc1_eia = pathlib.Path().cwd().parent / 'ferc1_to_eia.pkl.gz'
file_path_deprish_eia = pathlib.Path().cwd().parent / 'deprish_to_eia.pkl.gz'

In [8]:
inputs = connect_deprish_to_ferc1.DeprishToFERC1Inputs(
    file_path_mul=file_path_mul,
    file_path_steam_ferc1=file_path_steam_ferc1,
    file_path_ferc1_eia=file_path_ferc1_eia,
    file_path_deprish_eia=file_path_deprish_eia
)

In [9]:
connect_deprish_to_ferc1.connect(inputs)

Reading the master unit list from /Users/christinagosnell/code/rmi-ferc1-eia/master_unit_list.pkl.gz
Connected plants:    103
Not connected:       51
Missing connections: 3
Portion of depreciation records compared to FERC Steam are:
    Smaller:    40.78%
    Bigger:     1.95%
    Same level: 11.52%
    No link:    45.74%


In [ ]:
def get_simple_connections(connects_deprish_eia, connects_ferc1_eia):
    connects_clean_deprish_ferc1 = pd.merge(connects_deprish_eia, connects_ferc1_eia, 
                                      on=['record_id_eia'],
                                      suffixes=('_eia','_ferc1'),
                                      how='outer', indicator=True
                                     )

    connects_simple = connects_clean_deprish_ferc1[connects_clean_deprish_ferc1['_merge'] == 'both']
    logger.info(
        f"Simple Deprish to EIA Connections: {len(connects_clean)/len(connects_deprish_eia):.02%}")
    return connects_simple    